# Naive Bayes Classification

This notebook runs through a worked example of the naive Bayes classifier.

## Load Data

The task is name-gender prediction given a set of labeled names.

In [ ]:
import random
# data is name,gender,freq
data = [line.strip().split(',') for line in open('names.txt').readlines()[1:]] 
# remove infrequent names
data = [line for line in data if int(line[2])>=50]
random.shuffle(data)
names = [line[0].lower() for line in data]
genders = [line[1] for line in data]

print 'Loaded', len(data), 'names'
print 'Some of the names'
print names[:20]
print genders[:20]

## Design Features

Convert each name to a set of features that are true for that name.

In [ ]:
def name2feats(name):
    feats = set()
    if name[-1] in 'aeiou':
        feats.add('ends-vowel')
    if name[-1] == 'n':
        feats.add('ends-n')
    if name == name[::-1]:
        feats.add('palindrome')
    if name[-1] == 'a':
        feats.add('ends-a')
    if name[-1] == 'o':
        feats.add('ends-o')
    if name[0] in 'aeiou':
        feats.add('starts-vowel')
    return feats
        
featurized_names = map(name2feats, names)

splitpoint = int(.8*len(genders))
train_names, test_names = names[:splitpoint], names[splitpoint:]
train_fnames, test_fnames = featurized_names[:splitpoint], featurized_names[splitpoint:]
train_genders, test_genders = genders[:splitpoint], genders[splitpoint:]

## Training: Estimate Probabilities 

1. For each feature, estimate P(feature|gender)

2. For each gender, estimate P(gender)

from training data

In [ ]:
from collections import defaultdict
prob_feat_given_gender = defaultdict(lambda : defaultdict(float)) # nested dict
prob_gender = defaultdict(float)

for featurized_name, gender in zip(train_fnames, train_genders):
    for feat in featurized_name:
        prob_feat_given_gender[gender][feat] +=1
    prob_gender[gender] +=1
   
# normalize counts to get MLE probabilities
for gender in prob_feat_given_gender:
    total = sum(prob_feat_given_gender[gender].values())
    for feat in prob_feat_given_gender[gender]:
        prob_feat_given_gender[gender][feat]/=total
    
total = sum(prob_gender.values())
for gender in prob_gender:
    prob_gender[gender]/=total
   
# display probabilities
for gender in prob_feat_given_gender:
    print 'P({0}) = {1}'.format(gender, prob_gender[gender])
    for feat in prob_feat_given_gender[gender]:
        print 'P({0} given {1}) = {2}'.format(feat, 
                                              gender, 
                                              prob_feat_given_gender[gender][feat])


## Testing or Prediction

Given a name, compute 

1. P(name|gender) for each gender by multiplying P(feat|gender) for each feat in name
2. P(gender|name) using Bayes Rule
3. Pick gender that gives larger P(gender|name)

In [ ]:
def predict(featurized_name):
    prob_gender_given_name = {}
    for gender in prob_gender:
        # 1. P(name|gender)
        p = 1.
        for feat in featurized_name:
             p *= prob_feat_given_gender[gender][feat]
        # 2. value proportional to P(gender|name) (prediction trick in slide)
        prob_gender_given_name[gender] = p*prob_gender[gender]
    return max(prob_gender_given_name.items(), key=lambda x:x[1])[0]

for name, featurized_name, gender in zip(test_names[:50], test_fnames, test_genders):
    print name, 'predicted', predict(featurized_name), 'actually', gender 
